<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

This notebook is modified based on https://github.com/microsoft/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [1]:
%%capture --no-stderr
# %pip install pyautogen~=0.2.0b4

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
import autogen
import os

config_list_gpt4 = [
    {
        "model": "gpt-4",
        "api_key": os.getenv("OPENAI_API_KEY"),
        # "api_key": str(os.environ["OPENAI_API_KEY"]),
    },
    {
        "model": "gpt-4-32k",
        "api_key": os.getenv("OPENAI_API_KEY"),
    },
]


# config_list_gpt35 = autogen.config_list_from_json(
#     "OAI_CONFIG_LIST",
#     filter_dict={
#         "model": {
#             "gpt-3.5-turbo",
#             "gpt-3.5-turbo-16k",
#             "gpt-3.5-turbo-0301",
#             "chatgpt-35-turbo-0301",
#             "gpt-35-turbo-v0301",
#         },
#     },
# )

/home/panoevj/anaconda3/envs/automemgpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Group Chat

In [3]:
import random
from typing import List, Dict
from autogen.agentchat.groupchat import GroupChat
from autogen.agentchat.agent import Agent
from autogen.agentchat.assistant_agent import AssistantAgent


class CustomGroupChat(GroupChat):
    def __init__(self, agents, messages, max_round=10):
        super().__init__(agents, messages, max_round)
        self.previous_speaker = None  # Keep track of the previous speaker

    def select_speaker(self, last_speaker: Agent, selector: AssistantAgent):
        # Check if last message suggests a next speaker or termination
        last_message = self.messages[-1] if self.messages else None
        if last_message:
            if "NEXT:" in last_message["content"]:
                suggested_next = last_message["content"].split("NEXT: ")[-1].strip()
                print(f"Extracted suggested_next = {suggested_next}")
                try:
                    return self.agent_by_name(suggested_next)
                except ValueError:
                    pass  # If agent name is not valid, continue with normal selection
            elif "TERMINATE" in last_message["content"]:
                try:
                    return self.agent_by_name("User_proxy")
                except ValueError:
                    pass  # If 'User_proxy' is not a valid name, continue with normal selection

        team_leader_names = [
            agent.name for agent in self.agents if agent.name.endswith("1")
        ]

        if last_speaker.name in team_leader_names:
            team_letter = last_speaker.name[0]
            possible_next_speakers = [
                agent
                for agent in self.agents
                if (
                    agent.name.startswith(team_letter)
                    or agent.name in team_leader_names
                )
                and agent != last_speaker
                and agent != self.previous_speaker
            ]
        else:
            team_letter = last_speaker.name[0]
            possible_next_speakers = [
                agent
                for agent in self.agents
                if agent.name.startswith(team_letter)
                and agent != last_speaker
                and agent != self.previous_speaker
            ]

        self.previous_speaker = last_speaker

        if possible_next_speakers:
            next_speaker = random.choice(possible_next_speakers)
            return next_speaker
        else:
            return None

In [4]:
# Termination message detection
def is_termination_msg(content) -> bool:
    have_content = content.get("content", None) is not None
    if have_content and "TERMINATE" in content["content"]:
        return True
    return False

## Construct Agents

In [5]:
llm_config = {"config_list": config_list_gpt4}

In [6]:
agents_sales = [
    AssistantAgent(
        name="Sales1",
        system_message="You are the team leader of the sales team, your team consists of Sales2 and Sales3. You can talk to the other team leaders: Marketing1, CustomerSupport1, ProductDevelopment1. Use 'NEXT: <agent_name>' to suggest the next speaker.",
        llm_config=llm_config,
    ),
    AssistantAgent(
        name="Sales2",
        system_message="You are team member Sales2, you are responsible for communication with ClientA.",
        llm_config=llm_config,
    ),
    AssistantAgent(
        name="Sales3",
        system_message="You are team member Sales3, you are responsible for communication with ClientB.",
        llm_config=llm_config,
    ),
]

agents_marketing = [
    AssistantAgent(
        name="Marketing1",
        system_message="You are the team leader of the marketing team, your team consists of Marketing2 and Marketing3. You can talk to the other team leaders: Sales1, CustomerSupport1, ProductDevelopment1. Use 'NEXT: <agent_name>' to suggest the next speaker.",
        llm_config=llm_config,
    ),
    AssistantAgent(
        name="Marketing2",
        system_message="You are team member Marketing2, you are responsible for generating non-digital marketing material.",
        llm_config=llm_config,
    ),
    AssistantAgent(
        name="Marketing3",
        system_message="You are team member Marketing3, you are responsible for digital marketing.",
        llm_config=llm_config,
    ),
]

agents_product_development = [
    AssistantAgent(
        name="ProductDevelopment1",
        system_message="You are the team leader of the product development team, your team consists of ProductDevelopment2 and ProductDevelopment3. You can talk to the other team leaders: Sales1, CustomerSupport1, ProductDevelopment1. Use 'NEXT: <agent_name>' to suggest the next speaker.",
        llm_config=llm_config,
    ),
    AssistantAgent(
        name="ProductDevelopment2",
        system_message="You are team member ProductDevelopment2, you are responsible for the development of the product.",
        llm_config=llm_config,
    ),
    AssistantAgent(
        name="ProductDevelopment3",
        system_message="You are team member ProductDevelopment3, you are responsible for quality assurance.",
        llm_config=llm_config,
    ),
]

agents_customer_support = [
    AssistantAgent(
        name="CustomerSupport1",
        system_message="You are the team leader of the customer support team, your team consists of CustomerSupport2 and CustomerSupport3. You can talk to the other team leaders: Sales1, Marketing1, ProductDevelopment1. Use 'NEXT: <agent_name>' to suggest the next speaker.",
        llm_config=llm_config,
    ),
    AssistantAgent(
        name="CustomerSupport2",
        system_message="You are team member of the customer support team, you are responsible for customer support for ClientA.",
        llm_config=llm_config,
    ),
    AssistantAgent(
        name="CustomerSupport3",
        system_message="You are team member of the customer support team, you are responsible for customer support for ClientB.",
        llm_config=llm_config,
    ),
]


agents_managers = [
    AssistantAgent(
        name="Manager1",
        system_message="You are the manager of the company, responsible of managing and coordinating the activities of all teams and departments within the company. You can talk to all team leaders: Sales1, Marketing1, CustomerSupport1, ProductDevelopment1. Use 'NEXT: <agent_name>' to suggest the next speaker.",
        llm_config=llm_config,
    ),
]

In [7]:
# Terminates the conversation when TERMINATE is detected.
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="Terminator admin.",
    code_execution_config=False,
    is_termination_msg=is_termination_msg,
    human_input_mode="NEVER",
)

In [8]:
list_of_agents = (
    agents_sales + agents_marketing + agents_product_development + agents_managers
)
list_of_agents.append(user_proxy)

## Start GroupChat

In [9]:
# Create CustomGroupChat
group_chat = CustomGroupChat(
    agents=list_of_agents,  # Include all agents
    messages=[
        'Everyone cooperate and and solve the various incoming tasks and client requests. Team Sales has Sales1, Sales2, Sales3. Team Marketing has Marketing1, Marketing2, Marketing3. Team ProductDevelopment has ProductDevelopment1, ProductDevelopment2, ProductDevelopment3. Team CustomerSupport has CustomerSupport1, CustomerSupport2, CustomerSupport3. Team Managers has Manager1. Only members of the same team can talk to one another. Only team leaders (names ending with 1) can talk amongst themselves. You must use "NEXT: Sales1" to suggest talking to Sales1 for example; You can suggest only one person, you cannot suggest yourself or the previous speaker; You can also not suggest anyone.'
    ],
    max_round=20,
)

In [10]:
# Create the manager
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config)

In [11]:
# Initiates the chat with the manager
agents_managers[0].initiate_chat(
    manager,
    message="ClientA, who recently purchased an e-bike from our company, has reported that after only a few weeks of regular use, the e-bike's battery began to malfunction. Please investigate and resolve the issue.",
)

Manager1 (to chat_manager):

ClientA, who recently purchased an e-bike from our company, has reported that after only a few weeks of regular use, the e-bike's battery began to malfunction. Please investigate and resolve the issue.

--------------------------------------------------------------------------------
Marketing2 (to chat_manager):

As Marketing2, I handle non-digital marketing materials such as brochures, flyers, and print ads. The issue you've described involves customer service and technical assistance. A team member from Customer Support or Technical Service team is better suited to handle this kind of problem. However, I'll pass this information to the relevant department.

--------------------------------------------------------------------------------
Marketing1 (to chat_manager):

NEXT: CustomerSupport1

--------------------------------------------------------------------------------
Extracted suggested_next = CustomerSupport1
ProductDevelopment1 (to chat_manager):

Hi